In [2]:
import pygame
from mpmath import mp
import numpy as np
from scipy.stats import linregress
import random
import sys

# Initialize Pygame
pygame.init()

# Set up display
info = pygame.display.Info()
width, height = info.current_w, info.current_h
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("El Examen")

# Colors
white = (255, 255, 255)
gray = (211, 211, 211)
black = (0, 0, 0)
blue = (0,0,255)
red = (255,0,0)

#Font
font = pygame.font.Font(None,32)
font_2 = pygame.font.Font(None,25)

#Buttons
button_width, button_height = int(width/7), 50

button_color = black
button_math = pygame.Rect(int(width/7), 150, button_width, button_height)
button_chem = pygame.Rect(3*int(width/7), 150, button_width, button_height)
button_eng = pygame.Rect(5*int(width/7), 150, button_width, button_height)


dropdown = pygame.Rect(-10,-20, width/4, height/20) 
home_button = pygame.Rect(50,-30,30,30)
close_button = pygame.Rect(10,-30,30,30)

# Create a clock object to control the frame rate
clock = pygame.time.Clock()
fps = 60  # Set the desired frames per second
running = True

def dropdown_dynamics(root, dropdown, home_button, close_button):
    mouse_x, mouse_y = pygame.mouse.get_pos()

    hover = dropdown.collidepoint(mouse_x, mouse_y)
    if hover and dropdown.height < 80 and home_button.y < 10:
        dropdown.height += 5
        home_button.y += 5
        close_button.y += 5 
    elif not hover and dropdown.height > 40 and home_button.y >-30:
        dropdown.height -= 5
        home_button.y -= 5
        close_button.y -= 5

    dropdown = pygame.Rect(-10,-20, 200, dropdown.height) 
    pygame.draw.rect(root, gray, dropdown, 0, 20)

    home_button = pygame.Rect(50, home_button.y,30,30)
    pygame.draw.rect(root, black, home_button,0, 15)
    home_text = font.render("[]", True, white)
    home_text_1 = home_text.get_rect(center=home_button.center)
    root.blit(home_text, home_text_1)

    close_button = pygame.Rect(10, close_button.y,30,30)
    pygame.draw.rect(root, black, close_button,0, 15)
    close_text = font.render("X", True, white)
    close_text_1 = close_text.get_rect(center=close_button.center)
    root.blit(close_text, close_text_1)

def button_dynamics(button, name, subname_1, subname_2, subname_3):
        hover = button.collidepoint(mouse_x, mouse_y)

        if hover and button.height < 350:
            button.height += 10
        elif not hover and button.height > 50:
            button.height -= 15
        
        button = pygame.Rect(button.x, button.y, button.width, button.height)
        pygame.draw.rect(screen, black, button,0, 30)

        text = font.render(name, True, white)
        text_button = text.get_rect(center=(button.centerx, button.y+25))
        screen.blit(text, text_button)

        subbutton_1 = subbutton_2 = subbutton_3 = None

        if button.height > 110:
            subbutton_1 = pygame.Rect(button.x+10,button.y+60,button_width-20, button_height)
            pygame.draw.rect(screen, white, subbutton_1,0, 30)
            text_1 = font_2.render(subname_1, True, black)
            text_subbutton_1 = text_1.get_rect(center=subbutton_1.center)
            screen.blit(text_1, text_subbutton_1)

        if button.height > 210:
            subbutton_2 = pygame.Rect(button.x+10,button.y+160,button_width-20, button_height)
            pygame.draw.rect(screen, white, subbutton_2,0, 30)
            text_2 = font_2.render(subname_2, True, black)
            text_subbutton_2 = text_2.get_rect(center=subbutton_2.center)
            screen.blit(text_2, text_subbutton_2)

        if button.height > 310:
            subbutton_3 = pygame.Rect(button.x+10,button.y+260,button_width-20, button_height)
            pygame.draw.rect(screen, white, subbutton_3,0, 30)
            text_3 = font_2.render(subname_3, True, black)
            text_subbutton_3 = text_3.get_rect(center=subbutton_3.center)
            screen.blit(text_3, text_subbutton_3)
        
        return subbutton_1, subbutton_2, subbutton_3

def generate_points_with_correlation(R, n_points=100):
    # Generate random x values
    x = np.random.randn(n_points)
    
    # Generate random noise
    noise = np.random.randn(n_points)
    
    # Generate y values that will have the desired correlation with x
    y = R * x + np.sqrt(1 - R**2) * noise
    
    return x, y

def Math_Game_Uno():
    global running
    math_game_uno = pygame.display.set_mode((width, height))
    pygame.display.set_caption('Math uno')
    clock = pygame.time.Clock()
    fps = 60  # Set the desired frames per second

    # Define colors
    white = (255, 255, 255)
    black = (0, 0, 0)
    blue = (0, 0, 255)
    red = (255, 0, 0)

    # Define font
    font = pygame.font.Font(None, 36)

    # Define input box
    input_box = pygame.Rect(width-640, height-600, 140, 32)
    color_inactive = pygame.Color('gray64')
    color_active = pygame.Color('gray17')
    color = color_inactive
    active = False
    text = '0.'

    # Initialize the correlation coefficient
    R = random.randint(0, 100)/100
    # Generate the points
    x, y = generate_points_with_correlation(R)

    # Define guess button
    guessing = True
    guess_button = pygame.Rect(width-500, height-600, 100, 32)
    # Define next button
    next_button = pygame.Rect(width-640, height-600, 120, 32)
    # Define True R box 
    true_R_box = pygame.Rect(width-640, height-550, 120, 32)
    # Define Guessed R box
    guessed_R_box = pygame.Rect(width-640, height-500, 120, 32)
    # Define difference box
    difference_box = pygame.Rect(width-640, height-450, 120, 32)
    # Define points box
    points_box = pygame.Rect(width-1330, height-720, 120, 32)
    current_points = 0

    life_slots = 3

    while True:
        math_game_uno.fill(white)
        mouse_x, mouse_y = pygame.mouse.get_pos()

        # Draw square outline of the graph around the middle of the window
        window_width, window_height = math_game_uno.get_size()
        graph_size = min(window_width, window_height) // 2
        graph_x = (window_width - graph_size) // 2 - 300
        graph_y = (window_height - graph_size) // 2
        pygame.draw.rect(math_game_uno, 'gray64', (graph_x, graph_y, graph_size, graph_size), 2)
        pygame.draw.line(math_game_uno, 'gray64', (graph_x, graph_y + graph_size // 2), (graph_x + graph_size, graph_y + graph_size // 2), 2)
        pygame.draw.line(math_game_uno, 'gray64', (graph_x + graph_size // 2, graph_y), (graph_x + graph_size // 2, graph_y + graph_size), 2)
        

        # Plot newly generated points
        plot_points_with_pygame(x, y, math_game_uno, graph_x, graph_y, graph_size)

        # Draw the points box
        pygame.draw.rect(math_game_uno, white, points_box)
        points_text = font.render("Points: "+ str(current_points), True, black)
        points_text_rect = points_text.get_rect(center=points_box.center)
        math_game_uno.blit(points_text, points_text_rect)

        # Draw life slots
        slot_width = 50
        slot_height = 20
        for i in range(3):
            color_box = red if i < life_slots else black
            pygame.draw.rect(math_game_uno, color_box, (width-1350 + i * slot_width + slot_width // 2, height-770, slot_width - 10, slot_height))

        if guessing:
            # Draw input box
            pygame.draw.rect(math_game_uno, color, input_box, 3)
            txt_surface = font.render(text, True, color)
            input_box.w = max(200, txt_surface.get_width() + 10)
            math_game_uno.blit(txt_surface, (input_box.x + 5, input_box.y + 5))

            # Draw guess button
            pygame.draw.rect(math_game_uno, blue, guess_button)
            guess_text = font.render("Guess", True, white)
            guess_text_rect = guess_text.get_rect(center=guess_button.center)
            math_game_uno.blit(guess_text, guess_text_rect)
        else:
            # Draw next button
            pygame.draw.rect(math_game_uno, blue, next_button)
            next_text = font.render("Next", True, white)
            next_text_rect = next_text.get_rect(center=next_button.center)
            math_game_uno.blit(next_text, next_text_rect)

            # Draw True R box
            pygame.draw.rect(math_game_uno, white, true_R_box)
            true_R_text = font.render("True R: " + str(R), True, black)
            true_R_text_rect = true_R_text.get_rect(center=true_R_box.center)
            math_game_uno.blit(true_R_text, true_R_text_rect)

            # Draw Guessed R box
            pygame.draw.rect(math_game_uno, white, guessed_R_box)
            guessed_R_text = font.render("Guessed R: "+ str(text), True, black)
            guessed_R_text_rect = guessed_R_text.get_rect(center=guessed_R_box.center)
            math_game_uno.blit(guessed_R_text, guessed_R_text_rect)

            # Draw difference box
            pygame.draw.rect(math_game_uno, white, difference_box)
            difference_text = font.render("Difference: "+ str(abs(R - float(text)))[:4], True, black)
            difference_text_rect = difference_text.get_rect(center=difference_box.center)
            math_game_uno.blit(difference_text, difference_text_rect)

        dropdown_dynamics(math_game_uno, dropdown, home_button, close_button)

        for event in pygame.event.get():
            # Makes the dropdown_dynamics function work
            if home_button.collidepoint(mouse_x,mouse_y) and event.type == pygame.MOUSEBUTTONDOWN:
                return
            if close_button.collidepoint(mouse_x,mouse_y) and event.type == pygame.MOUSEBUTTONDOWN:
                running = False
                return
            if event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE:
                return
            
            # Primary code for the input box
            if event.type == pygame.MOUSEBUTTONDOWN and guessing:
                if input_box.collidepoint(event.pos):
                    active = not active
                else:
                    active = False

                if guess_button.collidepoint(event.pos):
                    try:
                        if float(text) <= 1:
                            active = False
                            guessing = False
                            if abs(R - float(text)) < 0.1:
                                current_points += 1
                            else:
                                life_slots -= 1
                        else:
                            text = '0.'
                            active = False
                    except:
                        print("fail")
                        text = '0.'
                        active = False

            color = color_active if active else color_inactive

            if event.type == pygame.KEYDOWN and active:
                if event.key == pygame.K_RETURN:
                    try:
                        if float(text) <= 1:
                            active = False
                            guessing = False
                            if abs(R - float(text)) < 0.1:
                                current_points += 1
                            else:
                                life_slots -= 1
                        else:
                            text = '0.'
                            active = False
                    except:
                        print("fail")
                        text = '0.'
                        active = False
                elif event.key == pygame.K_BACKSPACE:
                    text = text[:-1]
                else:
                    text += event.unicode
            
            if event.type == pygame.MOUSEBUTTONDOWN and not guessing:
                if next_button.collidepoint(event.pos):
                    R = random.randint(0, 100)/100
                    x, y = generate_points_with_correlation(R)

                    guessing = True
                    text = '0.'
                    active = False

        pygame.display.flip()
        # Cap the frame rate to 60 fps
        clock.tick(fps)

def plot_points_with_pygame(x, y, screen, graph_x, graph_y, graph_size):
    # Define colors
    # Scale the points to fit the screen
    x_scaled = (x - x.min()) / (x.max() - x.min()) * graph_size + graph_x
    y_scaled = height - (y - y.min()) / (y.max() - y.min()) * graph_size - graph_y
    
    # Calculate the regression line
    slope, intercept, r_value, p_value, std_err = linregress(x, y)
    x_min, x_max = x.min(), x.max()
    y_min = slope * x_min + intercept
    y_max = slope * x_max + intercept
    
    x_min_scaled = (x_min - x.min()) / (x.max() - x.min()) * graph_size + graph_x
    x_max_scaled = (x_max - x.min()) / (x.max() - x.min()) * graph_size + graph_x
    y_min_scaled = height - (y_min - y.min()) / (y.max() - y.min()) * graph_size - graph_y
    y_max_scaled = height - (y_max - y.min()) / (y.max() - y.min()) * graph_size - graph_y
       
    # Draw the points
    for i in range(len(x_scaled)):
        pygame.draw.circle(screen, black, (int(x_scaled[i]), int(y_scaled[i])), 3)
        
    # Draw the regression line
    pygame.draw.line(screen, red, (int(x_min_scaled), int(y_min_scaled)), (int(x_max_scaled), int(y_max_scaled)), 2)

def Math_Game_Dos():
    global running
    math_game_dos = pygame.display.set_mode((width, height))
    pygame.display.set_caption('Constant Runner')
    clock = pygame.time.Clock()
    fps = 60  # Set the desired frames per second

    text = font_2.render("Choose a constant", True, white)
    
    numberkeys ={
            pygame.K_1: "1",
            pygame.K_2: "2",
            pygame.K_3: "3",
            pygame.K_4: "4",
            pygame.K_5: "5",
            pygame.K_6: "6",
            pygame.K_7: "7",
            pygame.K_8: "8",
            pygame.K_9: "9",
            pygame.K_0: "0"
        }
    
    constants ={
        "π": "3.14159265358979323846264338327950288419716939937510582097494459230781640628620899862803482534211706798214808651328230664709384460955058223172535940812848111745028410270193852110555964462294895493038196",
        "e": "2.71828182845904523536028747135266249775724709369995957496696762772407663035354759457138217852516642742746639193200305992181741359662904357290033429526059563073813232862794349076323382988075319525101901",
        "φ": "1.61803398874989484820458683436563811772030917980576286213544862270526046281890244970720720418939113748475408807538689175212663386222353693179318006076672635443338908659593958290563832266131992829026788",
        "√2": "1.41421356237309504880168872420969807856967187537694807317667973799073247846210703885038753432764157273501384623091229702492483605585073721264412149709993583141322266592750559275579995050115278206057147",
        "ln(2)": "0.69314718055994530941723212145817656807550013436025525412068000949339362196969471560586332699641868754200148102057068573368552023575813055703267075163507596193072757082837143519030703862389167347112335" 
    }

    rects = []
    number_rects = []

    for i in range(5):
        x = 2*i*(width/15) + width/5
        rects.append((pygame.Rect(x,50,width/15,width/15),blue))
    
    box_width = width // 10
    box_height = width // 10
    spacing = width // 40
    start_x = (width - (8 * box_width + 7 * spacing)) // 2  # center the boxes
    y = 3 * height // 4

    character_image = pygame.transform.scale(pygame.image.load('Constant_runner.png'),(box_width//2, box_height*3//2))
    character_x = start_x + 2 * (box_width + spacing)
    character_y = y - box_height*3//2

    score_text_box = pygame.Rect(width-width/12, 50, width/12, width/15)
    
    for i in range(8):
        x = start_x + i * (box_width + spacing)
        rect = pygame.Rect(x, y, box_width, box_height)
        number_rects.append((rect, ""))

    max_locked_out = width//7
    n = 3

    constant_running = False
    locked_out = False
    locked_out_time = max_locked_out

    while True:
        math_game_dos.fill(white)
        
        character = pygame.Rect(character_x, character_y, box_width//2, box_height*3//2)

        mouse_x, mouse_y = pygame.mouse.get_pos()
        dropdown_dynamics(math_game_dos, dropdown, home_button, close_button)
        
        Game_text_box = pygame.Rect(width/2-100, height/2-100, 200, 200)
        if not constant_running:
            text_start = font_2.render("Choose a constant to start game", True, black)
            text_start_2 = text_start.get_rect(center=Game_text_box.center)
            math_game_dos.blit(text_start, text_start_2)

        for event in pygame.event.get():
            # Makes the dropdown_dynamics function work
            if home_button.collidepoint(mouse_x,mouse_y) and event.type == pygame.MOUSEBUTTONDOWN:
                return
            if close_button.collidepoint(mouse_x,mouse_y) and event.type == pygame.MOUSEBUTTONDOWN:
                running = False
                return
            if event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE:
                return

            # From here you can fuck around ~~~~~~~~~~~
            if event.type == pygame.MOUSEBUTTONDOWN:
                for i in range(len(rects)):
                    rect, color = rects[i]
                    if rects[i][0].collidepoint(mouse_x,mouse_y):
                        rects[i] = (rect,red)
                        number = list(constants.values())[i]
                        constant = list(constants.keys())[i]
                        n = 3
                        number_rects = []
                        for k in range(8):
                            x = start_x + k * (box_width + spacing)
                            rect = pygame.Rect(x, y, box_width, box_height)
                            number_rects.append((rect, ""))
    
                        number_rects[-2] = (number_rects[-2][0], number[0])
                        number_rects[-1] = (number_rects[-1][0], number[1])
                            
                        for j in range(len(rects)):
                            if j!= i:
                                rects[j]= (rects[j][0],blue)
                        character_x = start_x + 2 * (box_width + spacing)
                        character_y = y - box_height*3//2
                        constant_running = True
                        locked_out = False
            
            if event.type == pygame.KEYDOWN:
                if event.key in numberkeys and constant_running:
                    if numberkeys[event.key] == number[n-1] and not locked_out:
                        if len(number_rects) < 8:
                            new_rect = pygame.Rect(number_rects[-1][0].x + box_width + spacing, y, box_width, box_height)
                            number_rects.append((new_rect, number[n-1]))
                        else:
                            for i in range(len(number_rects)):
                                number_rects[i][0].x = start_x + (i-1) * (box_width + spacing)
                            
                            new_rect = pygame.Rect(start_x + (len(number_rects)-1) * (box_width + spacing), y, box_width, box_height)
                            number_rects.append((new_rect, number[n-1]))
                        n += 1
                    else:
                        locked_out = True
        
        for i in range(len(number_rects) - 1, -1, -1): # Itererer bagfra
            number_rect, number_text = number_rects[i]
            pygame.draw.rect(math_game_dos, black, number_rect)
            text = font_2.render(number_text, True, white)
            text_math_2 = text.get_rect(center=number_rect.center)
            math_game_dos.blit(text, text_math_2)
            if constant_running:
                number_rect.x -= 5
                if number_rect.x < -box_width:
                    number_rects.pop(i)

        
        for i in range(len(rects)):
            rect, color = rects[i]
            pygame.draw.rect(math_game_dos, color, rect)

            label = list(constants.keys())[i]
            text_surface = font.render(label, True, (white))
            text_rect = text_surface.get_rect(center=rect.center)
            math_game_dos.blit(text_surface, text_rect) 

            math_game_dos.blit(character_image, (character.x, character.y))

        if locked_out:
            
            locked_out_box = pygame.Rect(3*width//7, height//5, width//7, height//20)
            pygame.draw.rect(math_game_dos, black, locked_out_box)

            if locked_out_time >= 0:
                locked_out_time -= 4
                locked_out_bar = pygame.Rect(3*width//7 + 2, height//5 + 2, locked_out_time - 4, height//20 - 4)
                pygame.draw.rect(math_game_dos, red, locked_out_bar)
            else:
                locked_out = False
                locked_out_time = max_locked_out
        
        if character.x > number_rects[-1][0].x + box_width:
            character_y += 7
            constant_running = False
            for i in range(5):
                next_num_rect = pygame.Rect(number_rects[-1][0].x + (i+1) * (box_width + spacing), y ,box_width, box_height)
                pygame.draw.rect(math_game_dos, gray, next_num_rect)
                next_number = font_2.render(str(number[n-1+i]), True, black)
                next_number_text = next_number.get_rect(center=next_num_rect.center)
                screen.blit(next_number, next_number_text)

        score = font_2.render(f"Digits: {n-3}", True, black)
        score_text = score.get_rect(left=score_text_box.left, centery=score_text_box.centery)
        screen.blit(score, score_text)

        pygame.display.flip()
        pygame.display.update()
        
        # Cap the frame rate to 60 fps
        clock.tick(fps)

def Math_Game_Tres():
    button_3 = pygame.Rect(width/2-100, height/2-100, 200, 200)
    pygame.draw.rect(screen, black, button_3,0, 30)
    text_math_3 = font_2.render("Math Game 3", True, white)
    text_math_1_3 = text_math_3.get_rect(center=button_3.center)
    screen.blit(text_math_3, text_math_1_3)

def Eng_Game_Uno():  
    button_4 = pygame.Rect(width/2-100, height/2-100, 200, 200)
    pygame.draw.rect(screen, black, button_4,0, 30)
    text_eng_1 = font_2.render("English Game 1", True, white)
    text_eng_1_1 = text_eng_1.get_rect(center=button_4.center)
    screen.blit(text_eng_1, text_eng_1_1)

def Eng_Game_Dos(): 
    button_5 = pygame.Rect(width/2-100, height/2-100, 200, 200)
    pygame.draw.rect(screen, black, button_5,0, 30)
    text_eng_2 = font_2.render("English Game 2", True, white)
    text_eng_1_2 = text_eng_2.get_rect(center=button_5.center)
    screen.blit(text_eng_2, text_eng_1_2)

def Eng_Game_Tres(): 
    button_6 = pygame.Rect(width/2-100, height/2-100, 200, 200)
    pygame.draw.rect(screen, black, button_6,0, 30)
    text_eng_3 = font_2.render("English Game 3", True, white)
    text_eng_1_3 = text_eng_3.get_rect(center=button_6.center)
    screen.blit(text_eng_3, text_eng_1_3)

def Chem_Game_Uno(): 
    button_7 = pygame.Rect(width/2-100, height/2-100, 200, 200)
    pygame.draw.rect(screen, black, button_7,0, 30)
    text_chem_1 = font_2.render("Chemistry Game 1", True, white)
    text_chem_1_1 = text_chem_1.get_rect(center=button_7.center)
    screen.blit(text_chem_1, text_chem_1_1) 

def Chem_Game_Dos(): 
    button_8 = pygame.Rect(width/2-100, height/2-100, 200, 200)
    pygame.draw.rect(screen, black, button_8,0, 30)
    text_chem_2 = font_2.render("Chemistry Game 2", True, white)
    text_chem_1_2 = text_chem_2.get_rect(center=button_8.center)
    screen.blit(text_chem_2, text_chem_1_2) 

def Chem_Game_Tres(): 
    button_9 = pygame.Rect(width/2-100, height/2-100, 200, 200)
    pygame.draw.rect(screen, black, button_9,0, 30)
    text_chem_3 = font_2.render("Chemistry Game 3", True, white)
    text_chem_1_3 = text_chem_3.get_rect(center=button_9.center)
    screen.blit(text_chem_3, text_chem_1_3) 


while running:
    screen.fill(white)
    mouse_x, mouse_y = pygame.mouse.get_pos()

    dropdown_dynamics(screen, dropdown, home_button, close_button)

    subbutton_math_1, subbutton_math_2, subbutton_math_3 = button_dynamics(button_math, 'Math', 'Game 1', 'Game 2', 'Game 3')
    subbutton_chem_1, subbutton_chem_2, subbutton_chem_3 = button_dynamics(button_chem, 'Chemistry', 'Game 1', 'Game 2', 'Game 3')
    subbutton_eng_1, subbutton_eng_2, subbutton_eng_3 = button_dynamics(button_eng, 'English', 'Game 1', 'Game 2', 'Game 3')
    
    for event in pygame.event.get():
        if event.type == pygame.MOUSEBUTTONDOWN:
            if close_button.collidepoint(mouse_x, mouse_y):
                running = False
            if subbutton_math_1 and subbutton_math_1.collidepoint(mouse_x, mouse_y):
                Math_Game_Uno()
            if subbutton_math_2 and subbutton_math_2.collidepoint(mouse_x, mouse_y):
                Math_Game_Dos()
            if subbutton_math_3 and subbutton_math_3.collidepoint(mouse_x, mouse_y):
                Math_Game_Tres()
            if subbutton_chem_1 and subbutton_chem_1.collidepoint(mouse_x, mouse_y):
                Chem_Game_Uno()
            if subbutton_chem_2 and subbutton_chem_2.collidepoint(mouse_x, mouse_y):
                Chem_Game_Dos()
            if subbutton_chem_3 and subbutton_chem_3.collidepoint(mouse_x, mouse_y):
                Chem_Game_Tres()
            if subbutton_eng_1 and subbutton_eng_1.collidepoint(mouse_x, mouse_y):
                Eng_Game_Uno()
            if subbutton_eng_2 and subbutton_eng_2.collidepoint(mouse_x, mouse_y):
                Eng_Game_Dos()
            if subbutton_eng_3 and subbutton_eng_3.collidepoint(mouse_x, mouse_y):
                Eng_Game_Tres()
                
    # Update the display
    pygame.display.flip()

    # Cap the frame rate to 60 fps
    clock.tick(fps)

# Quit Pygame
pygame.quit()
sys.exit()

SystemExit: 

C:\Users\kjaer\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
